In [1]:
# Generelle moduler og funksjonsbeskrivelser brukt i forelesningen
from numpy import sin, cos, pi, exp
import numpy.fft as fft
import numpy as np
import matplotlib.pyplot as plt
from Kildekode._10_Konvolusjon import *

%matplotlib ipympl

<img src="NTNU_Logo.png" align="left" style="width: 30%">
<br clear="all" />
<br></br>

# Digitale Filtre, Impulsrespons og Konvolusjon

* **Emne IELEA2302 - Signalbehandling**
* **Uke 9/10, 2021**
* **Relevant Pensum:**
    * Kapittel 5 i læreboka DSP First
* **Underviser: Kai Erik Hoff**

# Tema tirsdag 2. mars:
* Signal og System
* Digitale Filtre - hva og hvorfor?
* Bruk av digitale filtre

#### Repetisjon
* Eksempeloppgaver


# Tema fredag 12. mars:
* Repetisjon Signal og System
* Kausale LTI-system
* Impulsrespons
* FIR filtre
* Konvolusjon
* Sammensatte system

# Signal og System

### Generell terminologi

* **Signal:**
    * En funksjon som formidler informasjon om et fenomen.
* **System:**
    * Tar imot inngangssignal og produserer et nytt utgangssignal
    
    
* Eksempel på system:
<img src="Figurer/10_Digitale Filtre Intro/Fig0_DSPSystem.png" style="width: 80%; margin-left=100px" />

# LTI-system

* Vi skal i hovedsak begrense oss til å snakke om system som er ***lineære*** og ***tidsinvariante***.
    * Slike filtre omtales som ***LTI-system***
    * Det mest sentrale kjennetegnet på LTI-system er at sinusformede inngangssignal ***alltid*** vil gi sinusformede utgangssignal.
        * Utgangssignalet kan dog ha en annen *Amplitude* og *Fase*
    * *Mer om disse egenskapene i neste uke*

# Hva er et filter?
* I signalbehandlingssammenheng refererer et **filter** til et system som demper uønskede frekvenskomponenter i et signal.

    
<img src="Figurer/10_Digitale Filtre Intro/Fig2_FiltResp.png" style="width: 50%; margin-left=200px" />

# Analoge filtre
* Fysiske elektroniske kretser.
* Beskrives matematisk med en differensiallikning *(eller en transferfunksjon)*.
* Eksempel: RLC-krets:
<img src="Figurer/10_Digitale Filtre Intro/Fig1_AnalogFilt.png" style="width: 80%; margin-left=100px" />

# Digitale Filtre

* *Algoritme* for å regne ut hver sample av utgangssignalet $y[n]$
* Eksisterer kun som programvare
* Utgangssignalet er en ***lineær kombinasjon*** av signalverdier ved ulike tidspunkt
* Eksempel: *3-punkts middelverdifilter*
$$ y[n] = \frac{1}{3}\cdot x[n] + \frac{1}{3}\cdot x[n-1] + \frac{1}{3}\cdot x[n-2]$$
    * Utregning av sample nummer 5:
    $$y[5] = \frac{1}{3}\cdot x[5] + \frac{1}{3}\cdot x[4] + \frac{1}{3} \cdot x[3]$$
* Krever at programmet fører en "logg" over tidligere signalverdier

# Buksområder

* Bildebehandling
* Audioprosessering
* Kommunikasjonssystem
* Digitale kontrollere
* Mye mer

# Fordeler og ulemper

<br>
<div style="width: 100%;">
    <div style="float: left; width: 50%">
    <ul>
        <b> Fordeler </b>
        <br>
        <li> Tillater filterresponser som ikke kan oppnås med analoge filtre </li>
        <li> Kostnadseffektivt </li>
        <li> Enkle å modifisere </li>
    </ul>
    </div>
    <div style="float: right; width: 50%">
    <ul>
        <b> Ulemper</b>
        <br>
        <li> Kan tilføre ekstra forsinkelse i systemet </li>
        <li> Fungerer kun innenfor et frekvensbånd gitt av samplingsfrekvensen </li>
    </ul>
    </div>
</div> 

# Differanseligning

* Matematisk beskrivelse av et digitalt filter basert på ***filterkoeffisienter*** $a_k$ og $b_k$.
* Beskriver forholdet mellom inn- og utgangssignal.

### $$ \sum_{k=0}^{N}a_k\cdot  y[n-k] = \sum_{k=0}^{M} b_k \cdot x[n-k]$$

* Digital filterdesign sentrerer seg rundt å finne passende verdier for filterkoeffisientene $a_k$ og $b_k$. 
    * Disse verdiene anvendes så *direkte* i filteralgoritmen uten behov for å balansere kretskomponenter.

# Hvordan bruke digitale filter

1. Regn ut filterkoeffisientene $a_k$ og $b_k$.
    * Her er det vanlig å bruke dataverktøy som Python eller Matlab
    * Det finnes mange ulike fremgangsmåter avhengig av bruk
    
    
2. Implementer filteralgoritmen på en digital enhet
    * Dette kan for eksempel være i form av et program på en Arduino mikrokontroller.
    

## Eksempel: Regn ut filterparametre i Python

In [2]:
import scipy.signal as sig


# Kode for Digitalt Fjerdeordens Butterworth Lavpassfilter
b, a = sig.butter(N = 5,              # Filterorden
                  Wn = 0.35,          # Digital Knekkfrekvens (x pi)
                  btype = 'lowpass',  # Lavpassfilter
                  output = 'ba'       # Returner filterkoeffisenter
                  )

print("b:", b, "\na:", a)

# Vis frekvensresponsen til filteret
# (frekvensrespons o.l. vil utforskes i større detalj senere.)
w, H = sig.freqz(b, a)
plt.close(1); plt.figure(1)
plt.plot(w/pi, abs(H))

b: [0.01290369 0.06451846 0.12903692 0.12903692 0.06451846 0.01290369] 
a: [ 1.         -1.47967143  1.40369177 -0.68085641  0.1918841  -0.0221299 ]


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

## Eksempel: Implementasjon på Arduino
<pre>
<font color="#95a5a6">&#47;* A program to read from analog input periodically, filter the input using a digital butterworth filter,</font>
<font color="#95a5a6"> &nbsp;&nbsp;&nbsp;and print the filtered signal samples to the serial monitor. *&#47;</font>
<font color="#5e6d03">#include</font> <font color="#005c5f">&#34;IIR_filter.h&#34;</font> &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;<font color="#434f54">&#47;&#47; Contains IIR filter class</font>
<font color="#5e6d03">#include</font> <font color="#005c5f">&#34;Timer.h&#34;</font> &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;<font color="#434f54">&#47;&#47; Contains Timer class</font>

<font color="#00979c">const</font> <font color="#00979c">int</font> <font color="#000000">SIG_INPUT</font> <font color="#434f54">=</font> <font color="#000000">A0</font><font color="#000000">;</font> &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;<font color="#434f54">&#47;&#47; Signal Input Pin</font>
<font color="#00979c">const</font> <font color="#00979c">int</font> <font color="#000000">SIG_OUTPUT</font> <font color="#434f54">=</font> <font color="#000000">3</font><font color="#000000">;</font> &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;<font color="#434f54">&#47;&#47; Signal Output Pin</font>
<font color="#00979c">const</font> <font color="#00979c">unsigned</font> <font color="#00979c">int</font> <font color="#000000">T_S</font> <font color="#434f54">=</font> <font color="#000000">10</font><font color="#000000">;</font> &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;<font color="#434f54">&#47;&#47; Sample period</font>

<font color="#000000">IIR_filter</font> <font color="#000000">LP_Filt</font><font color="#000000">;</font> &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;<font color="#434f54">&#47;&#47; Create a filter object</font>
<font color="#000000">Timer</font> <font color="#000000">sampleTimer</font><font color="#000000">;</font> &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;<font color="#434f54">&#47;&#47; Timer to manage sampling intervals</font>

<font color="#00979c">void</font> <font color="#5e6d03">setup</font><font color="#000000">(</font><font color="#000000">)</font> <font color="#000000">{</font>
 &nbsp;<font color="#d35400">pinMode</font><font color="#000000">(</font><font color="#000000">SIG_INPUT</font><font color="#434f54">,</font> <font color="#00979c">INPUT</font><font color="#000000">)</font><font color="#000000">;</font>
 &nbsp;<font color="#d35400">pinMode</font><font color="#000000">(</font><font color="#000000">SIG_OUTPUT</font><font color="#434f54">,</font> <font color="#00979c">OUTPUT</font><font color="#000000">)</font><font color="#000000">;</font>

 &nbsp;<font color="#95a5a6">&#47;* Initialize myFilter object with filter coefficients and filter order as specified below *&#47;</font>
 &nbsp;<font color="#00979c">float</font> <font color="#000000">b</font><font color="#000000">[</font><font color="#000000">5</font><font color="#000000">]</font> <font color="#434f54">=</font> <font color="#000000">{</font><font color="#000000">0.03048527</font><font color="#434f54">,</font> <font color="#000000">0.12194108</font><font color="#434f54">,</font> <font color="#000000">0.18291162</font><font color="#434f54">,</font> &nbsp;<font color="#000000">0.12194108</font><font color="#434f54">,</font> <font color="#000000">0.03048527</font><font color="#000000">}</font><font color="#000000">;</font>
 &nbsp;<font color="#00979c">float</font> <font color="#000000">a</font><font color="#000000">[</font><font color="#000000">5</font><font color="#000000">]</font> <font color="#434f54">=</font> <font color="#000000">{</font><font color="#000000">1.</font><font color="#434f54">,</font> &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;<font color="#434f54">-</font><font color="#000000">1.17512564</font><font color="#434f54">,</font> <font color="#000000">0.92564873</font><font color="#434f54">,</font> <font color="#434f54">-</font><font color="#000000">0.31042133</font><font color="#434f54">,</font> <font color="#000000">0.04766256</font><font color="#000000">}</font><font color="#000000">;</font>
 &nbsp;<font color="#000000">LP_Filt</font><font color="#434f54">.</font><font color="#000000">initialize</font><font color="#000000">(</font><font color="#000000">b</font><font color="#434f54">,</font> <font color="#000000">a</font><font color="#000000">)</font><font color="#000000">;</font>
<font color="#000000">}</font>

<font color="#00979c">void</font> <font color="#5e6d03">loop</font><font color="#000000">(</font><font color="#000000">)</font> <font color="#000000">{</font>
 &nbsp;<font color="#5e6d03">if</font> <font color="#000000">(</font><font color="#000000">sampleTimer</font><font color="#434f54">.</font><font color="#000000">hasExpired</font><font color="#000000">(</font><font color="#000000">)</font><font color="#000000">)</font>
 &nbsp;<font color="#000000">{</font>
 &nbsp;&nbsp;&nbsp;<font color="#000000">sampleTimer</font><font color="#434f54">.</font><font color="#000000">start</font><font color="#000000">(</font><font color="#000000">T_S</font><font color="#000000">)</font><font color="#000000">;</font> &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
 &nbsp;&nbsp;&nbsp;<font color="#00979c">float</font> <font color="#000000">input_sample</font> <font color="#434f54">=</font> <font color="#000000">(</font><font color="#00979c">float</font><font color="#000000">)</font><font color="#d35400">analogRead</font><font color="#000000">(</font><font color="#000000">SIG_INPUT</font><font color="#000000">)</font><font color="#000000">;</font> &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;<font color="#434f54">&#47;&#47; Get Input Sample</font>
 &nbsp;&nbsp;&nbsp;<font color="#00979c">float</font> <font color="#000000">output_sample</font> <font color="#434f54">=</font> <font color="#000000">LP_Filt</font><font color="#434f54">.</font><font color="#000000">filter_sample</font><font color="#000000">(</font><font color="#000000">input_sample</font><font color="#000000">)</font><font color="#000000">;</font> &nbsp;&nbsp;&nbsp;&nbsp;<font color="#434f54">&#47;&#47; Filter latest input sample</font>
 &nbsp;&nbsp;&nbsp;<font color="#d35400">analogWrite</font><font color="#000000">(</font><font color="#000000">SIG_OUTPUT</font><font color="#434f54">,</font> <font color="#d35400">round</font><font color="#000000">(</font><font color="#000000">output_sample</font><font color="#434f54">&#47;</font><font color="#000000">4</font><font color="#000000">)</font><font color="#000000">)</font><font color="#000000">;</font> &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;<font color="#434f54">&#47;&#47; Set output level</font>
 &nbsp;<font color="#000000">}</font>
<font color="#000000">}</font>

</pre>

# Blokkskjema

<img src="Figurer/10_Digitale Filtre Intro/Fig9_Blocks.png" style="width: 80%" />

# Kausale LTI-system

* Vikgige egenskaper i signalbehandling
* Realiserbare system med stor grad av fleksibilitet

# Nøkkelegenskap: Kausalitet
* Aksjon $\rightarrow$ Reaksjon
* Kausale systemer avhenger kun av nåværende signalverdi og input til systemet som allerede har skjedd.
* Absolutt nødvendig for systemer som skal operere i sanntid (f.eks. digitale kontrollere).
    * Alle fysiske system er kausale.

* Systemer som prosesserer f.eks. komplette mediefiler/datafiler kan være antikausale.

* Når vi arbeider med hele datafiler i øvingene mister kausalitet som begrep noe av betydningen, men dersom man skal implementere et digitalt filter på f.eks. en arduino er det helt sentralt.

# Kausalitet illustrert

* "Fortids-vinduet" vokser med 1 sample for hver nye sample vi tar.
<img src="Figurer/10_Digitale Filtre Intro/Fig3_Kausalitet.png" style="width: 90%; margin-left=100px" />

## Eksempel: Kausalt 3-punkts middelverdifilter

* Bruker foregående sampleverdier for $x[n]$
    - Dersom filteret opererer i sanntid, er ikke neste sampleverdi tilgjengelig før det aktuelle samplingstidspunktet.

<img src="Figurer/10_Digitale Filtre Intro/Fig4_Kausalitet2.png" style="width: 100%" />

## 3-punkts middelverdifilter illustrert

<img src="Figurer/10_Digitale Filtre Intro/Fig5_Kausalitet3.png" style="width: 80%" />

## Eksempel: Antikausalt 3-pukts middelverdifilter

<img src="Figurer/10_Digitale Filtre Intro/Fig6_Kausalitet4.png" style="width: 100%" />

# Nøkkelegenskap: Linearitet

* Et system kan regnes som lineært dersom det oppfyller to enkle krav:
* Gitt $y[n] = F(x[n])$, der $F$ er funksjonen til systemet:

    * **Additivitet:**
    $$F(x_1[n] + x_2[n]) = F(x_1[n])+F(x_2[n])$$
    
    * **Homogenitet:**
    $$F(\alpha \cdot x[n]) = \alpha \cdot F(x[n])$$

## Illustrasjon av Additivet:


<img src="Figurer/10_Digitale Filtre Intro/Fig7_Additivity.png" style="width: 60%; margin-left: 100px%" />

# Nøkkelegenskap: Tidsinvarians


* Gitt $y[n] = F(x[n])$, der $F$ er funksjonen til systemet:
$$F(x[n-N]) = y[n-N]$$

* Illustrasjon:
<img src="Figurer/10_Digitale Filtre Intro/Fig8_TimeInv.png" style="width: 80%; margin-left: 100px%" />

## Eksempler:
* Eksempel på kausalt LTI system:
$$y[n] = 0.5\cdot x[n] - 0.5\cdot x[n-1]$$
* Eksempel på ulineært men tidsinvariant system:
$$y[n] = x^2[n]$$
* Eksempel på lineært men *ikke* tidsinvariant system:
$$y[n] = x[2\cdot n]$$

1. $y[n] = 2+x[n-1]$
2. $y[n] = n\cdot x[n]$
3. $y[n] = x[n]+x[n+5]$

# Generell differanselikning for kausale LTI-system

* ***Alle*** kausale LTI-system kan beskrives med en generell differanselikning:

### $$ \sum_{k=0}^{N}a_k\cdot  y[n-k] = \sum_{k=0}^{M} b_k \cdot x[n-k]$$

* $a$ og $b$ er tallrekker som inneholder de ulike koeffisientene til systemet/filteret.
\begin{align}
a_k &\in \{a_0, a_1,\ldots , a_N\}\\
b_k &\in \{b_0, b_1, \ldots, b_M\}
\end{align}

# Impulsrespons

* Premiss for LTI-system:
    * *Dersom systemets reaksjon til en deltapuls $\delta[n]$ er kjent, er det mulig å predikere hvordan systemet vil reagere på alle mulige inngangssignal.*
* Utgangssignalet til et LTI-system gitt en impuls $\delta[n]$ som inngangssignal kalles impulsresponsen $h[n]$ til systemet.

<br>
<br>
<img src="Figurer/10_Digitale Filtre Intro/Fig10_hn.png" style="width: 60%; margin-left: 200px%" />

# Repetisjon deltapuls
* **Definisjon:**
$$
\delta[n] = 
\begin{cases}
1, n=0\\
0, n\neq 0
\end{cases}
$$

* Etthvert signal kan beskrives som en *lineær kombinasjon* av deltapulser:
\begin{align}
x[n] &=\ldots + x[-1] \cdot \delta[n+1] + x[0]\cdot \delta[n] + x[1] \cdot \delta[n-1] + \ldots \\
&= \sum_{k=-\infty}^{\infty} x[k]\cdot \delta[n-k]
\end{align}

# Utregning av utgangssignal:

* Impulsrespons:
$$h[n] = F(\delta[n])$$

* Arbitrært tgangssignal $y[n]$:
\begin{align}
y[n] &= F(x[n])\\
     &= F\left(\sum_{k=-\infty}^{\infty} x[k]\cdot \delta[n-k]\right)\\
     &= \sum_{k=-\infty}^{\infty} x[k]\cdot F(\delta[n-k])\\
     &= \sum_{k=-\infty}^{\infty} x[k]\cdot h[n-k]
\end{align}

# Konvolusjon

* Utgangssignalet til et LTI-system utregnes ved å utføre ***konvolusjon*** av inngangssignalet $x[n]$ og impulsresponsen $h[n]$.
    * Konvolusjon har en egen matematisk operator: $*$.

\begin{align}
y[n] &= \sum_{k=-\infty}^{\infty} x[k]\cdot h[n-k]\\
&= x[n]*h[n]
\end{align}

# FIR Filtre


* **FIR** - **F**inite **I**mpulse **R**esponse
* Fellesbetegnelse for filter som regner ut utgangsveriden $y[n]$ *kun* med basert på en lineær kombinasjon av verdiene til inngangssignalet $x[n]$ ved forskjellige sampletidspunkt.

### $$y[n] = \sum_{k=0}^{M} b_k \cdot x[n-k]$$

* Filterordenen til et FIR filter er gitt ved konstanten $M$
    * Avgjør hvor langt tilbake i tid et filter må "huske" når det regner ut den filtrerte utgangsveriden $y[n]$.

# Blokkskjemarepresentasjon

* Visuell fremstilling av en filteralgoritme
* Følgende generelle form for FIR filter

<img src="Figurer/10_Digitale Filtre Intro/Fig13_FIR_struct.png" style="width: 80%" />

* Såkalt *Direkte Form* implementasjon

# FIR filtre impulsrespons

\begin{align}
h[n] &= \sum_{k=0}^{M} b_k \cdot \delta[n-k]\\
     &= b_0 \cdot \delta[n] + b_1\cdot \delta[n-1] + \ldots + b_M\cdot \delta[n-M]
\end{align}

* Impulsresponsen til et FIR filter er et signal som gjengir filterkoeffisientene $b_k$
<img src="Figurer/10_Digitale Filtre Intro/Fig11_FIR.png" style="width: 80%; margin-left: 100px%" />

## Eksempel på FIR filter: *Middelverdifilter*

* Et $L$-punkts middelverdifilter rgner ut gjennomsnittsverdien av de $L$ foregående samplene til signalet $x[n]$.
* Fungerer som et lavpassfilter ved å glatte ut momentane endringer i signalet.
* Filterkoeffisienter:
$$b_k = \left[\frac{1}{L}, \frac{1}{L}, \ldots, \frac{1}{L}\right]$$

* Lite komplisert og mye brukt.

## Regneeksempel 1:

* Finn et uttrykk for impulsresponsen $h[n]$ til et 8-punkts middelverdifilter.

## Demo: Filtrering av signal med konvolusjon

In [15]:
n = np.arange(30); xn = np.cos(pi/3*n) # Sinussekvens
hn = np.ones(5)/5                      # 5-punkts middelverdifilter
convolutionDemo(xn, hn, fig_num=2);    #figsize=(10,7)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Output()

* Stegvis beskrivelse av konvolusjon visuelt:
    1. Tidsreverser (flip) impulsresponsen $h[n]$ rund $n=0$
    2. Forskyv den filppede impulsresponsen $h[-n]$ langs tidsaksen.
    3. Utfør punktvis multiplikasjon mellom flippet og forskjøvet impulsrespons med inngangssignalet.
    4. Summer sammen resultatet fra punkt 3.
* Anbefaler å prøve seg frem med ulike typer signal og impulsrespons. 

## Observasjoner.

* Lengde utgangssignal tilsvarer summen av lengden til inngangssignalet og impulsresponsen minus 1.
$$L_y = L_x + L_h -1$$


* Digitale filtre har en "responstid", og det filtrerte signalet vil ofte "henge etter" inngangssignalet.

# Utregning av konvolusjon

\begin{align}
y[n] &= \sum_{k=-\infty}^{\infty} x[k]\cdot h[n-k]\\
     &= x[0]\cdot h[n]+x[1]\cdot h[n-1] + x[2]\cdot h[n-2]\ldots 
\end{align}

1. Sett opp de ulikt forkjøvne impulsresponsene $h[n-k]$ 
2. Multipliser med hver av samplene til $x[k]$
3. Legg sammen alle impulsresponsene.
 
* Mest oversiktlig ved å bruke en tabell (*eksempel på neste slide*)

## Regneeksempel 2:
* Bruk konvolusjon til å finne stegresponsen $s[n]$ til et system der impulsresponsen er 
$$h[n] = \delta[n]-\delta[n-1]+2\delta[n-2]-\delta[n-3]+\delta[n-4]$$

# Konvolusjon med python

* Vi bruker `numpy`-funksjonen `convolve()`.

In [4]:
xn = np.ones(10) # x[n] = u[n]-u[n-10]
hn = np.array([1, -1, 2, -1, 1]) # Fra regneeksempel 2
yn = np.convolve(xn, hn)
plt.close(3);plt.figure(3)
plt.stem(yn)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

<StemContainer object of 3 artists>

# Konvolusjon egenskaper

1. Kommutativ 
\begin{align}
x[n]*y[n] &= y[n]*x[n] \\
\sum_{k=-\infty}^{\infty} x[k]\cdot y[n-k] &= \sum_{k=-\infty}^{\infty} y[k]\cdot x[n-k]
\end{align}


2. Assossiativ
$$(x[n]*y[n])*v[n] = x[n]*(y[n]*v[n])$$


3. Distributiv *(linearitet)*
$$(x[n] + y[n])*v[n] = x[n]*v[n] + y[n]*v[n]$$


4. Konvolusjon og fouriertransformasjon
$$ x[n]*y[n] \leftrightarrow X\left(e^{j\hat{\omega}} \right)\cdot Y\left(e^{j\hat{\omega}} \right)$$

<img src="Figurer/10_Digitale Filtre Intro/Fig12_LTI_par.png" style="width: 80%" />

<img src="Figurer/10_Digitale Filtre Intro/Fig12_LTI_cas.png" style="width: 80%" />

## Regneeksempel 3:
* Finn impulsresponsen til det samlede systemet $h_{tot}[n]$.

<img src="Figurer/10_Digitale Filtre Intro/Fig14_Ex3.png" style="width: 70%; margin-left: 100px" />

# Oppsummering

* Digitale filtre tar i bruk en lineær kombinasjon av tilgjengelige signalsampler for å regne ut en utgangsverdi $y[n]$ for hver sample.
* *Lineære*, og *Tidsinvariante* (LTI) system tillater mye fleksibilitet med tanke på rekkefølge av operasjoner, og kan beskrives med sin *impulsrespons* $h[n]$.
* Utgangssignalet $y[n]$ til et LTI-system utregnes ved *konvolusjon* ($y[n] = x[n]*h[n]$).
* *Kausale* system baserer seg kun på foregående signalsampler som allerede har "vert gjennom" systemet ved utregning av utgangsverdi $y[n]$. 
* *Filterordenen* for et digitalt filter forteller hvor langt bak i tid systemet må huske tidligere sampler.
* FIR filter er en undergruppe av LTI-system hvor kun impulsresponsen $h[n]$ korresponderer med filterkoeffisentene.
    * Kun definert med filterkoeffisenter $b_k$